In [1]:
#imports

%matplotlib inline
import pandas as pd
import numpy as np
import time
import requests
import bs4
import json
from flatten_json import flatten

In [2]:
# 20 requests every 1 seconds(s)
# 100 requests every 2 minutes(s)

apikey = "RGAPI-d2088fc5-5f70-4703-8360-03eabf53dcca"

In [3]:
# get challenger summonerId from league-exp-api
# 1.2 delay bc short list

req = requests.get("https://euw1.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/GRANDMASTER/I?page=1&api_key={}"
                   .format(apikey),time.sleep(1.2))
assert req.status_code == 200
data = req.json()
sumids = [d['summonerId'] for d in data]

In [4]:
sumids

['2EbXGQo0fLjOzJ1y74INwlLxZVGz4W9M7drgQyNHT5kVTgY',
 'otmzo6xtXvv4uqYNDOojcX2InwctgbR9CuMLOXYy_Zk5cCM',
 'B4vGi0UyejzzkmLc2oV42S36RYJwGIJjkFj3bsDeoZz59c0',
 'ipBA0xp7fvjCe4Lu0edYJEX81fQJVLE5z4YJ0k5-05T9W_Q',
 'mAXHoAbyKTixCwVn3T8hDLauiATdqb23DDTUU8lV3se77aQ',
 'H1PaV6wkvVZRRSHcSjkns3NBeYQhz2wlXOlqIH64FwMaq_rG',
 '4fJJrInNPQO6M3Mg4iNPgwMYAsECQS2xSAG0qHVSJ5u74FLa',
 '8KFnbLNDwR0J-Eq0mDfS3dpGqPcg8HbvBqkD2la4oejxwmDa',
 'XPqv44Zr6IJrEYvADLRWxsKt1LfZ_6tDCjSTOe6Ui9zztjEL',
 'qQKP01VwE5NVW7WjlKYedMhjehXwZGUZBQhxYDz3qMbHk8Nb',
 '9Nsny9lPbUJjO7udgULCXp660gqU7ExvBgNQB6fUUmPhw_0',
 'YV_QyMdNEkywejvbIhnXHqQBFOY-j9guw154v58AU4j-y7s',
 '5kP59z-u-F7F39P87rILoaOIWtXJ7-IrVs-txhEjeFjmOXs',
 '8lRQPmGB4WC7FHVi1a9PbWBmpdY2WcSuCId9rid8p3mCO9Cg',
 'VmUReZTaFJEW0YjlNouHSPW_5M0Zg4Jz7Y7g-I2ggEDzia4',
 'GflEihIk_WteXd_3w6341obKFaSD74hU3tqTfEnU-wSlf2Y',
 'gQCNDEsoJsHgO7BsEQYnbhfp0ix2esojFkLrOdqRZ4CixkI',
 'dONas3NhwHYfCmwze523bgyaLSFMwO9sbWxNypvE6OX0ofk',
 '1lLHBMD-QX4MfPFkk3-4QjHP_kiJ3y0cx1sXUpMLVI8_gXo',
 'kT2F

In [5]:
# get accountId via summonerId via summoner-api
# 1.2 delay bc short list

accids = []
for sumid in sumids:
    sumreq = requests.get("https://euw1.api.riotgames.com/lol/summoner/v4/summoners/{}?api_key={}"
                          .format(sumid,apikey),time.sleep(1.2))
    assert sumreq.status_code == 200
    accids.append(sumreq.json()['accountId'])

In [6]:
accids

['Imrz17_kBscl7LrhPXZ3pLLaNuCrPRHWjkhBYoc5t-qPFA',
 'JHD54I6GnDCm1kzVFX1aicPAIt7lT543hIsvcMhhJpjNyw',
 '7E--efWiAfnRuy307fOhU9C_ep8bNtDHsGUmjfe1XgmpAA',
 'Pi1CyjWXP2aGFy-QTqFHFv2RaKtaTsYvzzSCQy41tMgN1Q',
 'qXSzyoR0jmJlFJKrC5sU6Yg_gBLLGOar-M81B3pHgIK-6Go',
 '86ITMeBXg14wMrgWNBWfi06ejv1QHky60m8z_5Ffl1_Jcow-MYn21wny',
 'gbNyceDN6JAHULzv2hPOxHVQtHjnwL3PMmuJw5-R9A6j7I7ctIAU7e99',
 'vgen1eqhryWEiPi_8UbB4ULaGEAEPSSu2TwZ0LzoagsqsrH0IuO3umQ6',
 'ffWmEkITwPkDVUZWVx8lFtHMkCx4NSHLXVZBTXksaceRVdJiGr74swjA',
 'vsJ576dt9zqZrSLhjX05W1EkJHzBs78ZqkNNDDPU5lqyK9vUFcNhYuMa',
 'h7L06qys6rUc4IVIMwoEJQnaXIVvbv83zVwNC4PPmtAyyA',
 '-AXBawUY7lhMzTZkFl2B20sqGtp1YQEvxmyduZMYz6g_Dw',
 'FpE6PjhxpohLEVyFegIY1Hu9AcN_VXbbq1YuczQ_8AHIayM',
 'xbSFoaIIvl9YGy7rfQrpJp0_x_M1WQ4-T0CwKiu_t2mDx_kQzjpahkie',
 'XweherojDo3RswkDBXUMAnx31Epf-qFb9OqziG2Oeho0bg',
 'FCeUShMA3MAMjmQL2vdEYllVcVsukOQ25fdYJz99RY8WLg',
 'QMK-FWjsRszOH3Sc9Eni97kFXpNj-aNqF0_fh6QkNS5RnA',
 'tuiGInQY5C1zBAB3mYnCt44lPi-nFyPcxEKSKfrDJsag0w',
 'HWZn_oszMKEE_aEYrm

In [ ]:
# get game ids from match history via match-api
# 1.3 delay bc long list

gamids = []

for acc in accids:
    matlreq = requests.get("https://euw1.api.riotgames.com/lol/match/v4/matchlists/by-account/{}?api_key={}"
                           .format(acc,apikey),time.sleep(1.5))
    assert matlreq.status_code == 200
    gamids += [d['gameId'] for d in matlreq.json()['matches']]

In [12]:
matlreq.status_code

504

In [14]:
len(gamids)

15900

In [ ]:
# print original length, then correct overlap from shared games and print new length

print(len(gamids))

gamids = set(gamids)

print(len(gamids))

In [35]:
with open('euw_chall_gam_id_list.txt', 'r') as f:
    challgames = f.read().splitlines()

In [46]:
challgames =set([int(e) for e in challgames])

In [47]:
len(challgames)

10446

In [48]:
len(gamids)

14557

In [52]:
len(gamids - (challgames & gamids))

10724

In [53]:
gamids = gamids - (challgames & gamids)

In [54]:
# get individual match information from gamids list via match-api
# 1.5 delay bc long list and repeated issues with 

matchlist = []

for gam in gamids:
    gamreq = requests.get("https://euw1.api.riotgames.com/lol/match/v4/matches/{}?api_key={}"
                          .format(gam,apikey),time.sleep(1.3))
    
    i=0
    while gamreq.status_code != 200:
        gamreq = requests.get("https://euw1.api.riotgames.com/lol/match/v4/matches/{}?api_key={}"
                          .format(gam,apikey),time.sleep(1.3))
        i+=1
        if i == 5:
            break
            
    match = flatten(gamreq.json())
    matchlist.append(match)

In [55]:
gamreq.status_code

200

In [56]:
# check length of matchlist 

check = [len(m) for m in matchlist]
len(matchlist),max(check),min(check)

(10724, 1550, 133)

In [57]:
# create df from matches

df = pd.DataFrame(matchlist)

In [68]:
gmgames = list(df['gameId'])

In [69]:
with open('gm_gam_id_list.txt', 'w') as f:
    for item in gmgames:
        f.write("%s\n" % item)

In [61]:
len(set(list(df['gameId'])))

10724

In [63]:
compression_opts = dict(method='zip',
                        archive_name='gm.csv')  

In [64]:
df.to_csv('gm.zip', index=False,
          compression=compression_opts)